# The Impact of Lower Interest Rates in Brazilian Stock Market

*By Daniel Deutsch, José Lucas Barretto, Kevin kühl and Lucas Miguel Agrizzi*

In [2]:
import os
import time
from datetime import date

import matplotlib.pyplot as plt
import pandas as pd
import requests
import yfinance as yf
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [2]:
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Data Collection

In this stage, we want to collect the data necessary for our analysis. In our case, we must collect two kinds of data: Indexes of (which kind? why?) and stock information about the indexes.

## Sector Indexes

Indexes are basically .... (explicar o que são indexes e porque eles serão utilizados na nossa análise).

Given that, the group decided to understand the effects of the index rate on the following indexes (because we considered them the most influential in the brazillian market):

| Index &emsp;  | Index Name                         | Sector           |
|:--------------|:-----------------------------------|:-----------------|
| `ICON`        | Índice de Consumo                  | Cyclical consumption, non-cyclical consumption and health |
| `IEEX`        | Índice de Energia Elétrica &emsp;  | Electricity |
| `IFNC`        | Índice Financeiro                  | Financial intermediaries, miscellaneous financial services, pension and insurance |
| `IMAT`        | Índice de Materiais Básicos        | Basic materials |
| `IMOB`        | Índice Imobiliário                 | Real estate and civil construction |
| `INDX`        | Índice Industrial                  | Basic materials, industrial goods, cyclical consumption, non-cyclical consumption, information technology and health |
| `UTIL`        | Índice Utilidade Pública           | Electricity, water and sanitation and gas |

In [3]:
indexes = [
    { 'index': "ICON", 'url_code': "klDT04" },
    { 'index': "IEEX", 'url_code': "klFRVg" },
    { 'index': "IFNC", 'url_code': "klGTkM" },
    { 'index': "IMAT", 'url_code': "klNQVQ" },
    { 'index': "IMOB", 'url_code': "klNT0I" },
    { 'index': "INDX", 'url_code': "klORFg" },
    { 'index': "UTIL", 'url_code': "lVUSUw" }
]

years = [
    { 'year': "2021", 'url_code': "IwMjE" },
    { 'year': "2020", 'url_code': "IwMjA" },
    { 'year': "2019", 'url_code': "IwMTk" },
    { 'year': "2018", 'url_code': "IwMTg" },
    { 'year': "2017", 'url_code': "IwMTc" },
    { 'year': "2016", 'url_code': "IwMTY" },
    { 'year': "2015", 'url_code': "IwMTU" },
    { 'year': "2014", 'url_code': "IwMTQ" },
    { 'year': "2013", 'url_code': "IwMTM" },
    { 'year': "2012", 'url_code': "IwMTI" },
    { 'year': "2011", 'url_code': "IwMTE" },
    { 'year': "2010", 'url_code': "IwMTA" },
    { 'year': "2009", 'url_code': "IwMDk" },
    { 'year': "2008", 'url_code': "IwMDg" },
    { 'year': "2007", 'url_code': "IwMDc" },
    { 'year': "2006", 'url_code': "IwMDY" },
    { 'year': "2005", 'url_code': "IwMDU" },
    { 'year': "2004", 'url_code': "IwMDQ" },
    { 'year': "2003", 'url_code': "IwMDM" },
    { 'year': "2002", 'url_code': "IwMDI" },
    { 'year': "2001", 'url_code': "IwMDE" },
    { 'year': "2000", 'url_code': "IwMDA" },
    { 'year': "1999", 'url_code': "E5OTk" },
    { 'year': "1998", 'url_code': "E5OTg" }
]

url_sectors = "https://sistemaswebb3-listados.b3.com.br/indexStatisticsProxy/IndexCall/GetPortfolioDay/eyJpbmRleCI6I%siLCJsYW5ndWFnZSI6InB0LWJyIiwieWVhciI6Ij%sifQ=="

df = pd.DataFrame()
for i_id, index in enumerate(indexes):
    for y_id, year in enumerate(years):
        r = requests.get(url_sectors % (index['url_code'], year['url_code']), verify=False)
        try:
            print(f"\rProgress indexes {i_id+1}/{len(indexes)} years {y_id+1}/{len(years)} df_size {df.shape[0]}", end="")
            r = r.json()
        except Exception:
            print(f"Error with index {index['index']} year {year['year']}")
        else:
            if r['results']:
                for result in r['results']:
                    day = result['day']
                    for key, val in result.items():
                        if key != "day":
                            date = f"{year['year']}-{int(key[9:]):02d}-{day:02d}"
                            rate_value = float(val.replace('.', '').replace(',', '.')) if val else pd.NA
                            df_new = pd.DataFrame({ 'date': [date], 'index': [index['index']], 'rate_value': [rate_value] })
                            df = pd.concat([df, df_new], ignore_index=True)

df.dropna(inplace=True)
df.sort_values(by=['date', 'index', 'rate_value'], inplace=True, ignore_index=True)
df.to_csv("./datasets/sector_indexes.csv.zip")

Progress indexes 7/7 years 24/24 df_size 48360

### Volatility Index

In [66]:
df_vix = yf.download("^VIX", period='max', interval='1d')
df_vix.reset_index(level=0, inplace=True)
df_vix.rename(columns={'Adj Close': 'rate_value', 'Date': 'date'}, inplace=True)
df_vix = df_vix[:][['date', 'rate_value']]
df_vix['index'] = 'vix'
df_vix.sort_values(by=['date', 'index', 'rate_value'], inplace=True, ignore_index=True)

[*********************100%***********************]  1 of 1 completed


### Macroeconomic Indicators

#### Interest Rate (SELIC)

In [44]:
url_selic = "http://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=csv"
df_selic = pd.read_csv(
    url_selic, 
    delimiter=';',
    header=0, 
    names = ['date', 'rate_value'], 
    decimal=',',
    parse_dates=[0]
)
df_selic['index'] = 'selic'
df_selic.sort_values(by=['date', 'index', 'rate_value'], inplace=True, ignore_index=True)

#### Inflation Rate (IPCA)

In [36]:
url_ipca = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/all/p/all/d/v63%202,v69%202,v2266%2013,v2263%202,v2264%202,v2265%202?formato=json"

df_ipca = pd.read_json(url_ipca)
df_ipca.drop([0], inplace=True)

mask = (df_ipca['MN'] == '%')
df_ipca = df_ipca[mask]

df_ipca.rename(columns={'D3C': 'date'}, inplace=True)
df_ipca['date'] = pd.to_datetime(df_ipca['date'], format='%Y%m')
df_ipca['V'] = pd.to_numeric(df_ipca['V'])

mask = (df_ipca['D2N'] == 'IPCA - Variação mensal')
ipca_pct_change = df_ipca[mask][['date', 'V']].rename(columns={'V': 'rate_value'})
ipca_pct_change['index'] = 'ipca_pct'

mask = (df_ipca['D2N'] == 'IPCA - Variação acumulada no ano')
ipca_annual_cumulative = df_ipca[mask][['date', 'V']].rename(columns={'V': 'rate_value'})
ipca_annual_cumulative['index'] = 'ipca_cum'

df_ipca = pd.concat([ipca_pct_change, ipca_annual_cumulative], axis=0)
df_ipca.sort_values(by=['date', 'index', 'rate_value'], inplace=True, ignore_index=True)

# Data Analysis